In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof
from datetime import datetime
from numpy import array


     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 6.4 MB 8.9 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 1.7 MB/s 


In [2]:
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
  #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
  'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in','all_cryptocurrencies_us']
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["suggestion"],axis=1)
  y = data["suggestion"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def extract_data(df):
    rows = []
    for each in range(15,df.shape[0]-1):
        sugg = 0
        if df[each][3] > df[each][0]:
          sugg = 1
        row = [
                df[each-15][0],
                df[each-15][1],
                df[each-15][2],
                df[each-15][3],
                df[each-15][4],
                df[each-14][0],
                df[each-14][1],
                df[each-14][2],
                df[each-14][3],
                df[each-14][4],
                df[each-13][0],
                df[each-13][1],
                df[each-13][2],
                df[each-13][3],
                df[each-13][4],
                df[each-12][0],
                df[each-12][1],
                df[each-12][2],
                df[each-12][3],
                df[each-12][4],
                df[each-11][0],
                df[each-11][1],
                df[each-11][2],
                df[each-11][3],
                df[each-11][4],
                df[each-10][0],
                df[each-10][1],
                df[each-10][2],
                df[each-10][3],
                df[each-10][4],
                df[each-9][0],
                df[each-9][1],
                df[each-9][2],
                df[each-9][3],
                df[each-9][4],
                df[each-8][0],
                df[each-8][1],
                df[each-8][2],
                df[each-8][3],
                df[each-8][4],
                df[each-7][0],
                df[each-7][1],
                df[each-7][2],
                df[each-7][3],
                df[each-7][4],
                df[each-6][0],
                df[each-6][1],
                df[each-6][2],
                df[each-6][3],
                df[each-6][4],
                df[each-5][0],
                df[each-5][1],
                df[each-5][2],
                df[each-5][3],
                df[each-5][4],
                df[each-4][0],
                df[each-4][1],
                df[each-4][2],
                df[each-4][3],
                df[each-4][4],
                df[each-3][0],
                df[each-3][1],
                df[each-3][2],
                df[each-3][3],
                df[each-3][4],
                df[each-2][0],
                df[each-2][1],
                df[each-2][2],
                df[each-2][3],
                df[each-2][4],
                df[each-1][0],
                df[each-1][2],
                df[each-1][1],
                df[each-1][3],
                df[each-1][4],
                sugg
        ]
        rows.append(row)
    return rows
def row_scaler(df):
  scaler = MinMaxScaler(feature_range=(1,10))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  scaled = pd.DataFrame(scaler.fit_transform(df.T).T,dtype=object)
  scaled["suggestion"] = last_column
  return scaled
def column_scaler(df):
  scaler = MinMaxScaler(feature_range=(0,1))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  df_scaled = scaler.fit_transform(df.to_numpy())
  df_scaled = pd.DataFrame(df_scaled)
  df_scaled["suggestion"] = last_column
  return df_scaled
def stick_dfs(dfs):
  dataframe = dfs[0]
  for i in range(1,len(dfs)):
     dataframe = pd.concat([dataframe,dfs[i]], ignore_index = True)
  dataframe = dataframe.dropna()
  dataframe = dataframe.astype(float)
  return dataframe
def process(dfs): 
   fixed_dfs = []
   for df in dfs:
      df = np.array(df)
      df = extract_data(df)
      df = pd.DataFrame(df)
      df = row_scaler(df)
      #df = column_scaler(df)
      fixed_dfs.append(df)
   df = stick_dfs(fixed_dfs)
   return df   

In [ ]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]

symbols = get_crypto_syms()
#symbols = read_syms_from_txt()
dfs = []
for symbol in symbols:
           data = yf.download(symbol,period="MAX",interval="1d",progress=False)
           if data.empty :
             print("Passing...")
           else:
               dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
        if df.shape[0] > 16:
          dfs.append(df)

data = process(dfs)
xTrain, xTest, yTrain, yTest = spliting(data)
data


1 Failed download:
- XNO-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- VAL-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- ICP-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- XNO-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- VAL-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- ICP-GBP: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- XNO-GBP: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-GBP: No data found for this date range, symbol may be delisted
Passing.

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,suggestion
0,9.708322,10.000000,8.880661,9.457898,9.457898,9.404394,9.392881,4.330484,5.616964,5.616964,...,2.986224,1.114674,2.612565,2.612565,2.668457,1.919472,3.502134,2.196875,2.196875,0.0
1,10.000000,9.988091,4.751815,6.082483,6.082483,6.043243,6.616869,1.430638,2.903681,2.903681,...,3.895014,2.257992,2.544923,2.544923,2.589540,1.000000,2.476544,1.267970,1.267970,0.0
2,7.184074,7.752701,2.611670,4.071875,4.071875,4.057593,7.381001,3.501426,5.709952,5.709952,...,3.648461,2.184786,2.450421,2.450421,2.463273,1.000000,2.922314,1.197550,1.197550,0.0
3,5.828621,8.161547,5.438210,6.988524,6.988524,6.921770,7.275584,5.707016,6.133967,6.133967,...,5.031692,3.682289,3.820963,3.820963,3.852842,1.000000,4.238329,1.249999,1.249999,0.0
4,7.704645,7.968474,6.798834,7.117201,7.117201,7.134140,7.694696,7.041543,7.484495,7.484495,...,5.703673,3.288936,3.475353,3.475353,3.478481,1.000000,4.284520,2.980415,2.980415,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1760729,5.363178,6.522051,4.967420,4.982635,4.982635,4.976470,5.522044,4.501045,4.718588,4.718588,...,4.045360,3.310179,3.323132,3.323132,3.317280,1.000000,3.317280,1.052591,1.052591,1.0
1760730,4.976470,5.522044,4.501045,4.718588,4.718588,4.726390,5.266031,4.334689,4.866606,4.866606,...,3.317280,1.000000,1.052591,1.052591,1.054932,1.046896,2.213806,1.729095,1.729095,1.0
1760731,4.726390,5.266031,4.334689,4.866606,4.866606,4.869805,5.883314,4.146642,4.178086,4.178086,...,2.213806,1.046896,1.729095,1.729095,1.719731,1.596682,2.336309,2.175104,2.175104,0.0
1760732,4.869805,5.883314,4.146642,4.178086,4.178086,4.180584,5.195494,3.297382,5.014860,5.014860,...,2.336309,1.596682,2.175104,2.175104,2.162151,1.638894,2.658566,1.902318,1.902318,1.0


In [8]:
model = Sequential()


model.add(Dense(3000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(2000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 3000)              228000    
                                                                 
 dense_8 (Dense)             (None, 2000)              6002000   
                                                                 
 dense_9 (Dense)             (None, 1000)              2001000   
                                                                 
 dense_10 (Dense)            (None, 750)               750750    
                                                                 
 dense_11 (Dense)            (None, 250)               187750    
                                                                 
 dense_12 (Dense)            (None, 1)                 251       
                                                                 
Total params: 9,169,751
Trainable params: 9,169,751
No

In [9]:
model.fit(xTrain,yTrain,epochs=50,batch_size=10000,validation_data=(xTest,yTest))

Epoch 1/50
159/159 [==============================] - 50s 291ms/step - loss: 0.7590 - accuracy: 0.5237 - val_loss: 0.6867 - val_accuracy: 0.5455
Epoch 2/50
159/159 [==============================] - 46s 288ms/step - loss: 0.6856 - accuracy: 0.5480 - val_loss: 0.6858 - val_accuracy: 0.5454
Epoch 3/50
159/159 [==============================] - 46s 286ms/step - loss: 0.6823 - accuracy: 0.5579 - val_loss: 0.6848 - val_accuracy: 0.5531
Epoch 4/50
159/159 [==============================] - 46s 291ms/step - loss: 0.6769 - accuracy: 0.5690 - val_loss: 0.6789 - val_accuracy: 0.5639
Epoch 5/50
159/159 [==============================] - 45s 286ms/step - loss: 0.6768 - accuracy: 0.5677 - val_loss: 0.6870 - val_accuracy: 0.5401
Epoch 6/50
159/159 [==============================] - 46s 286ms/step - loss: 0.6666 - accuracy: 0.5847 - val_loss: 0.6590 - val_accuracy: 0.5966
Epoch 7/50
159/159 [==============================] - 46s 287ms/step - loss: 0.6488 - accuracy: 0.6071 - val_loss: 0.6512 - val_ac

In [ ]:
model.save("9420_BinaryTest_v09.h5")

NameError: ignored

In [6]:
from google.colab import drive
drive.mount("/content/drive")

data = pd.read_csv("/content/drive/MyDrive/BinaryTest_v08_testing01.csv")
data = data.drop("Unnamed: 0",axis=1)
xTrain, xTest, yTrain, yTest = spliting(data)
data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(1584660, 75) (1584660,)
(176074, 75) (176074,)


,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,suggestion
0,9.708322,10.000000,8.880661,9.457898,9.457898,9.404394,9.392881,4.330484,5.616964,5.616964,...,2.986224,1.114674,2.612565,2.612565,2.668457,1.919472,3.502134,2.196875,2.196875,0.0
1,10.000000,9.988091,4.751815,6.082483,6.082483,6.043243,6.616869,1.430638,2.903681,2.903681,...,3.895014,2.257992,2.544923,2.544923,2.589540,1.000000,2.476544,1.267970,1.267970,0.0
2,7.184074,7.752701,2.611670,4.071875,4.071875,4.057593,7.381001,3.501426,5.709952,5.709952,...,3.648461,2.184786,2.450421,2.450421,2.463273,1.000000,2.922314,1.197550,1.197550,0.0
3,5.828621,8.161547,5.438210,6.988524,6.988524,6.921770,7.275584,5.707016,6.133967,6.133967,...,5.031692,3.682289,3.820963,3.820963,3.852842,1.000000,4.238329,1.249999,1.249999,0.0
4,7.704645,7.968474,6.798834,7.117201,7.117201,7.134140,7.694696,7.041543,7.484495,7.484495,...,5.703673,3.288936,3.475353,3.475353,3.478481,1.000000,4.284520,2.980415,2.980415,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1760729,5.363178,6.522051,4.967420,4.982635,4.982635,4.976470,5.522044,4.501045,4.718588,4.718588,...,4.045360,3.310179,3.323132,3.323132,3.317280,1.000000,3.317280,1.052591,1.052591,1.0
1760730,4.976470,5.522044,4.501045,4.718588,4.718588,4.726390,5.266031,4.334689,4.866606,4.866606,...,3.317280,1.000000,1.052591,1.052591,1.054932,1.046896,2.213806,1.729095,1.729095,1.0
1760731,4.726390,5.266031,4.334689,4.866606,4.866606,4.869805,5.883314,4.146642,4.178086,4.178086,...,2.213806,1.046896,1.729095,1.729095,1.719731,1.596682,2.336309,2.175104,2.175104,0.0
1760732,4.869805,5.883314,4.146642,4.178086,4.178086,4.180584,5.195494,3.297382,5.014860,5.014860,...,2.336309,1.596682,2.175104,2.175104,2.162151,1.638894,2.658566,1.902318,1.902318,1.0


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

data.to_csv("BinaryTest_v08_testing01.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
